# CWL CommandLineTool for stage-in

## Goal

Use a stage-in CWL workflow to stage a Landsat-9 acquisition


This notebook is linked to: https://eoap.github.io/mastering-app-package/cwl-workflow/stage-in/

## Setup

In [ ]:
export WORKSPACE=/workspace/mastering-app-package
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
rm -fr ${RUNTIME}/*
cd ${RUNTIME}

## Stage-in Python code

The CWL CommandLineTool for stage-in includes inline Python code.

Let's inspect that code,

It reads a STAC Item reference and stages the STAC Item and Assets and creates a STAC Catalog

In [ ]:
cat ${WORKSPACE}/cwl-cli/stage-in.cwl | yq e .requirements.InitialWorkDirRequirement.listing[0].entry -

The full stage-in CWL CommandLineTool is shown below.


The `InitialWorkDirRequirement` CWL requirement allows creating files at runtime.


In [ ]:
cat ${WORKSPACE}/cwl-cli/stage-in.cwl | yq e . -

## Build the stage-in container

In [ ]:
podman \
    build \
    --format docker \
    -t localhost/stage:latest \
    ${WORKSPACE}/water-bodies/command-line-tools/stage

## Stage-in a Landsat-9 acquisition

In [ ]:
ls9_ref="https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2/items/LC09_L2SP_042033_20231015_02_T1"

cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-cli/stage-in.cwl \
    --reference \
    ${ls9_ref} > staged.json

Check what was staged

In [ ]:
ls -l $( cat ${WORKSPACE}/runs/staged.json | jq -r .staged.location - | sed 's|file://||g' )/catalog.json

In [ ]:
stac describe  $( cat ${WORKSPACE}/runs/staged.json | jq -r .staged.location - | sed 's|file://||g' )/catalog.json

In [ ]:
tree $( cat ${WORKSPACE}/runs/staged.json | jq -r .staged.location - | sed 's|file://||g' )